In [312]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

import torch
import numpy as np

from method.learned_cn_method import CNModule, VecCNModule
from data import load_network
from util import Params

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [222]:
params = Params("../../experiments/disgenet/bio-pathways/dpp-10/learned_cn/lr_1e-2_ones/params.json")
_, adj, _ = load_network("../../" + params.ppi_network)

IOError: [Errno 2] No such file or directory: '../../experiments/disgenet/bio-pathways/dpp-10/learned_cn/lr_1e-2_ones/params.json'

In [220]:
cn_module = CNModule(params, adj)
vec_cn_module = VecCNModule(params, adj)
cn_module.train = True
W = torch.rand_like(cn_module.W)
cn_module.W = torch.nn.Parameter(W) 
vec_cn_module.E = torch.nn.Parameter(W)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [192]:
M = 20
N = adj.shape[0]

X = (torch.rand((M,N)) > 0.5).float()

In [193]:
cn_data = cn_module(X).data

In [195]:
torch.sum(torch.abs(vec_cn_module(X).data - cn_data))

tensor(0.)

Confirming How Weight Vector should be Multiplied with Adjacency

In [205]:
A = (torch.rand((5,5)) > 0.5).float()
A

tensor([[0., 1., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 1., 0.],
        [1., 1., 0., 0., 1.],
        [1., 1., 1., 1., 0.]])

In [209]:
W = torch.tensor([1.,2.,3.,4.,5.]).view(5,1)
W

tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.]])

In [210]:
WA = torch.mul(W, A)
WA

tensor([[0., 1., 0., 1., 0.],
        [0., 0., 2., 0., 0.],
        [0., 0., 3., 3., 0.],
        [4., 4., 0., 0., 4.],
        [5., 5., 5., 5., 0.]])

In [211]:
torch.matmul(A, WA)

tensor([[4., 4., 2., 0., 4.],
        [0., 0., 3., 3., 0.],
        [4., 4., 3., 3., 4.],
        [5., 6., 7., 6., 0.],
        [4., 5., 5., 4., 4.]])

In [213]:
torch.mul(torch.matmul(A, A), W.view(1,5))

tensor([[1., 2., 3., 0., 5.],
        [0., 0., 3., 4., 0.],
        [1., 2., 3., 4., 5.],
        [1., 4., 6., 8., 0.],
        [1., 4., 6., 8., 5.]])

In [214]:
torch.matmul(A, torch.mul(W, A))

tensor([[4., 4., 2., 0., 4.],
        [0., 0., 3., 3., 0.],
        [4., 4., 3., 3., 4.],
        [5., 6., 7., 6., 0.],
        [4., 5., 5., 4., 4.]])

Learned CN Method 

In [304]:
params = Params("../../experiments/disgenet/bio-pathways/dpp-10/learned_cn/cn_weighting/vec_lr_1e-0_ones/params.json")

In [298]:
A = (np.random.rand(5,5) > 0.5).astype(float)
A

array([[0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 0.],
       [1., 0., 0., 0., 1.],
       [1., 1., 0., 1., 0.],
       [1., 0., 1., 1., 0.]])

In [305]:
vec_cn = VecCNModule(params, A)
scalar_cn = CNModule(params, A)

In [306]:
M = 1
N = A.shape[0]

X = (torch.rand((M,N)) > 0.5).float()
W = torch.rand(())
X

tensor([[0., 1., 1., 1., 1.]])

In [307]:
vec_cn(X)


torch.Size([1, 1, 5])


tensor([0.9444, 0.7083, 0.6528, 0.9861, 0.7083], grad_fn=<SqueezeBackward3>)

In [308]:
scalar_cn(X)

tensor([[0.9444, 0.7083, 0.6528, 0.9861, 0.7083]], grad_fn=<MmBackward>)

In [309]:
params.d = 8

In [313]:
vec_cn_deep = VecCNModule(params, A)

In [314]:
vec_cn_deep(X)

torch.Size([8, 1, 5])


tensor([-0.1489, -0.1963, -0.2074, -0.1405, -0.1963], grad_fn=<SqueezeBackward0>)